In [20]:
import yfinance as yf
import numpy as np
import pandas as pd
from pathlib import Path
import seaborn as sns
import hvplot.pandas
from pandas_datareader import data as pdr
import datetime as dt
%matplotlib inline

yf.pdr_override()

In [4]:
# import total production data for petroleum and alternative energy
total_energy_production = pd.read_csv("../Data/Total_Production_RAW.csv").reset_index()
total_energy_production.head(10)

,level_0,level_1,level_2,Report generated on: 02-08-2021 21:25:26
0,Series Key,INTL.4415-1-USA-QBTU.A,Series Key,INTL.4419-1-USA-QBTU.A
1,Series Name,Total energy production from petroleum and oth...,Series Name,"Total energy production from nuclear, renewabl..."
2,Units,QBTU,Units,QBTU
3,Frequency,A,Frequency,A
4,1980,20.474,1980,8.168
5,1981,20.412,1981,8.421
6,1982,20.458,1982,9.111
7,1983,20.528,1983,9.698
8,1984,21.082,1984,9.99
9,1985,21.196,1985,10.16


In [5]:
# drop the additional rows that do not contain data by year
total_energy_production.drop([0,1,2,3],inplace=True)
total_energy_production.head()

,level_0,level_1,level_2,Report generated on: 02-08-2021 21:25:26
4,1980,20.474,1980,8.168
5,1981,20.412,1981,8.421
6,1982,20.458,1982,9.111
7,1983,20.528,1983,9.698
8,1984,21.082,1984,9.99


In [6]:
# rename the columns to the correct names
total_energy_production.rename(columns={
    "level_0":"Year",
    "level_1":"Total petroleum energy production qBtu",
    "level_2":"Year2",
    "Report generated on: 02-08-2021 21:25:26":"Total alternative energy production qBtu"},inplace=True)
total_energy_production.head()

,Year,Total petroleum energy production qBtu,Year2,Total alternative energy production qBtu
4,1980,20.474,1980,8.168
5,1981,20.412,1981,8.421
6,1982,20.458,1982,9.111
7,1983,20.528,1983,9.698
8,1984,21.082,1984,9.99


In [7]:
# create separate dataframes
petroleum_production = total_energy_production.drop(columns=["Year2","Total alternative energy production qBtu"])
alternative_production = total_energy_production.drop(columns=["Year","Total petroleum energy production qBtu"]).rename(columns={"Year2":"Year"})

In [6]:
# check dtypes of columns to see if need to change
# petroleum_production.info()
# alternative_production.info()

In [8]:
# change dtypes for petroleum production dataframe
petroleum_production["Total petroleum energy production qBtu"] = petroleum_production["Total petroleum energy production qBtu"].astype("float")
petroleum_production["Year"] = petroleum_production["Year"].astype("int")
petroleum_production.set_index("Year",inplace=True)
petroleum_production.head()

,Total petroleum energy production qBtu
Year,
1980,20.474
1981,20.412
1982,20.458
1983,20.528
1984,21.082


In [9]:
# change dtypes for alternative production dataframe
alternative_production["Total alternative energy production qBtu"] = alternative_production["Total alternative energy production qBtu"].astype("float")
alternative_production["Year"] = alternative_production["Year"].astype("int")
alternative_production.set_index("Year",inplace=True)
alternative_production.head()

,Total alternative energy production qBtu
Year,
1980,8.168
1981,8.421
1982,9.111
1983,9.698
1984,9.990


In [37]:
# plot for yearly production for petroleum
petroleum_production.hvplot.line(title="Annual Energy Production from Petroleum",width=1200,height=400,grid=True)

:Curve   [Year]   (Total petroleum energy production qBtu)

In [38]:
# plot for yearly production of alternative energy
alternative_production.hvplot.line(title="Annual Energy Production from Alternative Energy",width=1200,height=400,grid=True)

:Curve   [Year]   (Total alternative energy production qBtu)

In [12]:
# create combined dataframe
combined_production = pd.concat([petroleum_production, alternative_production],axis="columns",join="inner")
combined_production.rename(columns={
    "Total petroleum energy production qBtu":"Production (Oil)",
    "Total alternative energy production qBtu":"Production (Alternative)"
},inplace=True)
combined_production.head()

,Production (Oil),Production (Alternative)
Year,,
1980,20.474,8.168
1981,20.412,8.421
1982,20.458,9.111
1983,20.528,9.698
1984,21.082,9.990


In [19]:
# create cleaned CSV
combined_production.to_csv("../Data/Total_Production_CLEAN.csv",index=False)

In [39]:
combined_prod_correlation = combined_production.corr()
combined_prod_correlation.head()

,Production (Oil),Production (Alternative)
Production (Oil),1.000000,0.079985
Production (Alternative),0.079985,1.000000


In [43]:
# plot correlation between production
combined_production.hvplot.scatter(width=1200,height=400,grid=True,y='Production (Oil)',x='Production (Alternative)',title="Production Correlation: Alternative vs. Oil")

:Scatter   [Production (Alternative)]   (Production (Oil))

In [18]:
# show percent change by year for each method of production
combined_production_pct = combined_production.pct_change()
combined_production_pct.dropna(inplace=True)
combined_production_pct.head()

,Production (Oil),Production (Alternative)
Year,,
1981,-0.003028,0.030975
1982,0.002254,0.081938
1983,0.003422,0.064428
1984,0.026988,0.030109
1985,0.005407,0.017017


In [25]:
# create stock dataframes for oil, alternative, and combined
tickers = ['BLX','CIG','ELP','PLUG','XOM','RDS-B','CVX','BP']
start = dt.datetime(1975,1,1)
end = dt.datetime(2021,1,1)

stock_prices = pdr.get_data_yahoo(tickers,start,end)['Adj Close']
stock_prices = stock_prices.reset_index()

def to_string(x):
        return str(x)

stock_prices['Date'] = stock_prices['Date'].apply(to_string).apply(lambda x: x.split('-')[0]) #! Works: From 1975
stock_prices = stock_prices.groupby('Date').last().loc['1999':'2020', :] #! Works: From 175
stock_prices.head()

[*********************100%***********************]  8 of 8 completed


,BLX,BP,CIG,CVX,ELP,PLUG,RDS-B,XOM
Date,,,,,,,,
1999,7.240282,22.191998,0.805270,20.055025,4.164179,282.500000,22.071880,21.356483
2000,11.100283,18.394651,0.523426,20.164093,3.893942,146.875000,22.928116,23.542824
2001,9.542374,18.381559,0.518594,22.035337,3.742501,87.400002,19.815769,21.754280
2002,1.547871,16.625727,0.273495,16.947132,1.367200,44.900002,19.266241,19.823975
2003,6.541880,21.011765,0.700814,22.940788,2.320834,72.500000,23.249554,23.915960


In [26]:
# create cleaned csv for stock prices
stock_prices.to_csv("../Data/Combined_Stock_Prices_CLEAN.csv", index=False)

In [32]:
# create separate dataframes for oil and alt prices, pct change, and returns

#prices dataframes
stock_prices_alt = stock_prices.drop(columns=['XOM','RDS-B','CVX','BP'])
stock_prices_oil = stock_prices.drop(columns=['BLX','CIG','ELP','PLUG'])

#returns dataframes
alt_returns = stock_prices_alt.pct_change()
oil_returns = stock_prices_alt.pct_change()

#portfolio returns dataframes
weights = [0.25,0.25,0.25,0.25]
alt_port_returns = pd.DataFrame(alt_returns.dot(weights)).dropna()  
oil_port_returns = pd.DataFrame(oil_returns.dot(weights)).dropna()
alt_port_returns.columns = ['Alternative Energy']
oil_port_returns.columns = ['Oil & Gas']

# combined portfolio returns
combined_portfolio_returns = pd.concat(
                                        [alt_port_returns, oil_port_returns], 
                                            axis = 'columns', 
                                                join = 'inner'
    )
combined_portfolio_returns.head()

,Alternative Energy,Oil & Gas
Date,,
2000,-0.090464,-0.090464
2001,-0.148352,-0.148352
2002,-0.607841,-0.607841
2003,1.525254,1.525254
2004,0.086424,0.086424


In [33]:
# create cumulative returns dataframe
cumulative_returns = (1 + combined_portfolio_returns).cumprod()
cumulative_returns.head()

,Alternative Energy,Oil & Gas
Date,,
2000,0.909536,0.909536
2001,0.774605,0.774605
2002,0.303768,0.303768
2003,0.767092,0.767092
2004,0.833387,0.833387


In [34]:
# create 5 year rolling standard deviation dataframe
df_rolling_std = combined_portfolio_returns.rolling(window = 5).std().dropna()* 100
df_rolling_std.head()

,Alternative Energy,Oil & Gas
Date,,
2004,80.877676,80.877676
2005,79.796062,79.796062
2006,77.326304,77.326304
2007,60.787798,60.787798
2008,23.535227,23.535227


In [35]:
# create total standard deviation by industry dataframe
stdev = pd.DataFrame(combined_portfolio_returns.std().sort_values())
stdev.columns = ['Standard Deviation']
stdev.head()

,Standard Deviation
Alternative Energy,0.654518
Oil & Gas,0.654518
